In [3]:
from EEGNet import *
from DeepConvNet import *
from dataloader import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [4]:
x_train, y_train, x_test, y_test = read_bci_data()

x_train = torch.from_numpy(x_train).float().to(device)
y_train = torch.from_numpy(y_train).type(torch.LongTensor).to(device)

x_test = torch.from_numpy(x_test).float().to(device)
y_test = torch.from_numpy(y_test).type(torch.LongTensor).to(device) # data type is long

(1080, 1, 2, 750) (1080,) (1080, 1, 2, 750) (1080,)


In [32]:
def test_model(model_name, EEGNet=False, DeepConvNet=False, relu=False, elu=False, lrelu=False):
    if EEGNet:
        if relu:
            model = EEGNet_relu().to(device)
        elif elu:
            model = EEGNet_elu().to(device)
        elif lrelu:
            model = EEGNet_lrelu().to(device)
        
    elif DeepConvNet:
        if relu:
            model = DeepConvNet_relu().to(device)
        elif elu:
            model = DeepConvNet_elu().to(device)
        elif lrelu:
            model = DeepConvNet_lrelu().to(device)
            
    model_load = torch.load(model_name, map_location=device)
    model_load.eval()

    
    test_loss, correct = 0, 0
    pred = model_load(x_test)
#     test_loss += loss_fn(pred, y_test).item()
    correct += (pred.argmax(1) == y_test).type(torch.float).sum().item()

    correct /= len(x_test)

    # print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    # print(model_load)
    # summary(model_load, (1, 2, 750))
    return correct*100

In [40]:
eegnet_relu = test_model("EEGNet_model_relu.pt",EEGNet=True, relu=True)
eegnet_lrelu = test_model("EEGNet_model_lrelu.pt",EEGNet=True, lrelu=True)
eegnet_elu = test_model("EEGNet_model_elu.pt",EEGNet=True, elu=True)
deepconvnet_relu = test_model("DeepConvNet_model_relu.pt",DeepConvNet=True, relu=True)
deepconvnet_lrelu = test_model("DeepConvNet_model_lrelu.pt",DeepConvNet=True, lrelu=True)
deepconvnet_elu = test_model("DeepConvNet_model_elu.pt",DeepConvNet=True, elu=True)

In [41]:
print("{:>15}{:>15}{:>15}{:>15}".format("", "ReLU", "Leaky ReLU", "ELU"))
print("{:>15}{:>15.6f}{:>15.6f}{:>15.6f}".format("EEGNet", eegnet_relu, eegnet_lrelu, eegnet_elu))
print("{:>15}{:>15.6f}{:>15.6f}{:>15.6f}".format("DeepConvNet", deepconvnet_relu, deepconvnet_lrelu, deepconvnet_elu))

                          ReLU     Leaky ReLU            ELU
         EEGNet      89.074074      88.240741      82.685185
    DeepConvNet      83.703704      83.425926      82.129630
